In [1]:
from sklearn.tree import DecisionTreeClassifier
from lightgbm.sklearn import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
import lightgbm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import scipy.io as sio

### 1.准备数据

In [13]:
dpath = "./data/"

In [14]:
train_final = pd.read_csv(dpath+"train_final.csv")
train_final.shape

(7377403, 21)

In [4]:
train_final.head()

,msno,song_id,source_system_tab,source_screen_name,source_type,city,bd,gender,registered_via,registration_init_time,...,song_length,genre_ids,language,mult_genre,user_pop,item_pop,user_rate,item_rate,lfm_reco,target
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,1,6,5,1,0,2,7,8,...,-0.25183,17,8,0,15.581,0.795,0.506,0.474,0.53734,1
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,3,7,3,13,4,0,9,7,...,0.23361,6,8,0,1.991,-0.081,0.743,1.000,0.99810,1
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,3,7,3,13,4,0,9,7,...,-0.13422,6,8,0,1.991,-0.072,0.743,0.500,-0.04898,1
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,3,7,3,13,4,0,9,7,...,0.05294,40,0,0,1.991,-0.081,0.743,1.000,0.99810,1
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,1,6,5,1,0,2,7,8,...,-0.36784,1,8,0,15.581,1.211,0.506,0.364,0.38750,1


In [15]:
X_train = np.array(train_final.iloc[:,2:-1])
y_train = np.array(train_final.iloc[:,-1])

### 2. xgboost提取特征

In [6]:
%%time
params = {'learning_rate': 0.05,
          'n_estimators': 10, 
          'max_depth': 5,
          'subsample': 0.6,
          'colsample_bytree':0.5,
          'nthread': 4,
         }
xgb = XGBClassifier(**params)
xgb.fit(X_train,y_train)

[23:07:17] Tree method is automatically selected to be 'approx' for faster speed. to use old behavior(exact greedy algorithm on single machine), set tree_method to 'exact'
Wall time: 45.9 s


In [13]:
#保存模型
pickle.dump(xgb,open("./model/xgb.pkl","wb"))

In [7]:
%%time
X_train_leaf = xgb.apply(X_train)
X_train_leaf.shape

Wall time: 7.17 s


In [12]:
#保存X_train_leaf
sio.mmwrite("./data/X_train_leaf", X_train_leaf)

In [4]:
X_train_leaf = sio.mmread("./data/X_train_leaf")

In [5]:
X_train_leaf.shape

(7377403, 10)

In [9]:
%%time
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
X_train_enc = enc.fit_transform(X_train_leaf)

D:\Program Files\Anaconda\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


Wall time: 6.61 s


In [12]:
X_train_enc.shape

(7377403, 296)

### 3.训练LR

In [16]:
%%time
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(n_jobs=4, penalty='l1')
lr.fit(X_train_enc,y_train)

D:\Program Files\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Program Files\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:1300: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 4.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


Wall time: 9min 39s


In [17]:
y_proba = lr.predict_proba(X_train_enc)
auc = roc_auc_score(y_train,y_proba[:,1])
print(auc)

0.7841150007717118


In [18]:
#保存预测结果
pickle.dump(y_proba,open("./model/xgb_lr_pred.pkl","wb"))